<center><h1>SALES ANALYSIS</h1></center>

In [515]:
import datetime
import pandas as pd
import os
import json
pd.options.mode.chained_assignment = None
year = datetime.datetime.today().year
print(year)
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

2020


In [514]:
config = json.loads(open("./config.json").read())
source_folder = config["sourceFolder"]
destination_folder = config["destinationFolder"]
check_extention = ['.csv']

def merge_files():
    download_path = ''
    for filename in os.listdir(source_folder):
        if any(ext in filename for ext in check_extention):
            #print(filename)
            download_path = (os.path.join(source_folder,filename))
            df = pd.read_csv(download_path, skiprows=1)
            df.to_csv(destination_folder, index=False, mode='a', header=False)
        else:
            print("Data folder is empty..")
    print("Files are merged..")
merge_files()            
df = pd.read_csv(destination_folder)

Data folder is empty..
Files are merged..


In [499]:
df1 = df.groupby(["Country"]).size().reset_index(name='Sales')
fig = go.Figure(data=go.Choropleth(
    locationmode = 'country names',
    locations = df1['Country'],
    z = df1['Sales'],
    text = df1['Country'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '<',
    colorbar_title = 'SALES',
))

fig.update_layout(
    title_text='SALES BY COUNTRY',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [500]:
df['Car_Make'].value_counts().iplot(kind='bar', 
                                    title='SALES BY CAR MAKE',
                                    xTitle='Car Make',yTitle='Customers',
                                    color='blue')

In [501]:
df1 = df.groupby(["Gender"]).size().reset_index(name='Sales')
fig = {'data' : [{'type' : 'pie',
                 'labels' : df1['Gender'],
                 'values' : df1['Sales'],
                 'direction' : 'clockwise',
                 'marker' : {'colors' : ["rgb(0,70,135)", "rgb(156,196,231)"]}}],
      'layout' : {'title' : 'SALES BY GENDER'}}

iplot(fig)

In [502]:
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import statsmodels.api as sm

In [503]:
data_main = df.groupby(["Date"]).size().reset_index(name='Sales')
data_main['Date'] = pd.to_datetime(data_main['Date'])
data_main.set_axis(data_main['Date'], inplace=True)
data_main.sort_index(inplace=True)
data1 = data_main.drop(columns=['Date'])

In [504]:
s = sm.tsa.seasonal_decompose(data1.Sales.values, freq=60)
trace1 = go.Scatter(x = np.arange(0, len(s.observed), 1),
                    y = s.observed,
                    mode = 'lines',
                    name = 'Observed',
                    line = dict(color = ('rgb(66, 134, 244)'), width = 2))
trace2 = go.Scatter(x = np.arange(0, len(s.trend), 1),
                    y = s.trend,
                    mode = 'lines',
                    name = 'Trend',
                    line = dict(color = ('rgb(255, 102, 102)'), width = 3))
trace3 = go.Scatter(x = np.arange(0, len(s.seasonal), 1),
                    y = s.seasonal,
                    mode = 'lines',
                    name = 'Seasonal',
                    line = dict(color = ('rgb(66, 244, 155)'), width = 2))
trace4 = go.Scatter(x = np.arange(0, len(s.resid), 1),
                    y = s.resid,
                    mode = 'lines',
                    name = 'Residual',
                    line = dict(color = ('rgb(255, 153, 51)'), width = 2))
data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'Seasonal Decomposition', xaxis = dict(title = 'Days'), yaxis = dict(title = 'Customers'))
fig = dict(data=data, layout=layout)
iplot(fig)

In [505]:
#Numpy Array
dataset = data1.values
training_data_len = math.ceil(len(dataset) * .8)
# Scale the Data
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(dataset)

In [506]:
# Creating Training Dataset
training_set = training_set_scaled[0:training_data_len, :]

# Creating a data structure with 25 timesteps and 1 output
x_train = []
y_train = []
for i in range(25, len(training_set)):
    x_train.append(training_set[i-25:i, 0])
    y_train.append(training_set[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
# Reshaping
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [507]:
# Initialising the RNN
model = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(x_train, y_train, epochs = 50, batch_size = 32, verbose = 0)
        

In [508]:
# Test Data
test_set = training_set_scaled[training_data_len - 25:,:]
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(25, len(test_set)):
    x_test.append(test_set[i-25:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
prediction = model.predict(x_test)
prediction = sc.inverse_transform(prediction)
train = data1[:training_data_len]
test = data1[training_data_len:]

In [509]:
date_train = data_main['Date'][:training_data_len]
date_test = data_main['Date'][training_data_len:]
train_plot = train.values.reshape((-1))
test_plot = test.values.reshape((-1))
test['Predictions'] = prediction
predict_plot = test.Predictions.values.reshape((-1))

In [510]:
trace1 = go.Scatter(
    x = date_train,
    y = train_plot,
    mode = 'lines',
    name = 'Training Data',
    line = dict(color=('rgb(66, 134, 244)'), width=2)
)
trace2 = go.Scatter(
    x = date_test,
    y = predict_plot,
    mode = 'lines',
    name = 'Predictions',
    line = dict(color=('rgb(255, 102, 102)'), width=2)
)
trace3 = go.Scatter(
    x = date_test,
    y = test_plot,
    mode = 'lines',
    name = 'Actuals',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    title = "Predictions",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Sales"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [511]:
# Lets predict next 25 Days
forecast_data = data1[-25:].values.reshape((-1))

def prediction(num_prediction, model):
    future_predictions = forecast_data[-25:]
    
    for _ in range(num_prediction):
        x = future_predictions[-25:]
        x_sc = sc.fit_transform(x.reshape((-1,1)))
        x_new = []
        x_new.append(x_sc)
        x_new = np.array(x_new)
        x_new = np.reshape(x_new,(1, 25, 1))
        value = model.predict(x_new)
        value_pred = sc.inverse_transform(value)
        future_predictions = np.append(future_predictions, value_pred)
    future_predictions = future_predictions[25-1:]
    return future_predictions

def prediction_date(num_prediction):
    last_date = data_main['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 25
forecast = prediction(num_prediction, model)
forecast_dates = prediction_date(num_prediction)

In [512]:
trace1 = go.Scatter(
    x = date_train,
    y = train_plot,
    mode = 'lines',
    name = 'Training Data',
    line = dict(color=('rgb(66, 134, 244)'), width=2)
)
trace2 = go.Scatter(
    x = date_test,
    y = predict_plot,
    mode = 'lines',
    name = 'Predictions',
    line = dict(color=('rgb(255, 102, 102)'), width=2)
)
trace3 = go.Scatter(
    x = date_test,
    y = test_plot,
    mode = 'lines',
    name = 'Actuals',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)
trace4 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Forecast',
    line = dict(color=('rgb(255, 153, 51)'), width=2)
)
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    title = "Predictions",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Sales"}
)
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

In [513]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code"></form>''')